## Note: 
Only run the cells below if you need to either build the sqlite database from scratch, or if you need to download new data. If you are downloading new data, make sure to put the database in the folder 'zip_data'.

If you don't have the database you need, go to https://drive.google.com/file/d/1J1TLVkvnEQvX31MxdF2df04BNpiaExyf/view?usp=sharing and then place it in the zip_data folder.

Don't forget to replace the link above with a new one that has the updated database!

Also be warned that constructing the database from scratch will take several hours!

In [1]:
import requests as r
import pandas as pd
import datetime
import requests as r
import db_zip_populator as zu
import re
import sqlite3
from tqdm import tqdm
import re

endyear = datetime.date.today().year
api_headers = {}
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'
db_path = 'zip_data/us_economic_data.db'


# Zipcode data


- The ZipCodeUtility class has 'startyear' and 'endyear' as parameters. If you need to download new data, you can adjust the start and end years. 

- There is a possibility that this won't work if the api call structure is changed. If that's the case, adjusting the '_get_response_data' function in db_zip_populator.py will do the trick.

In [ ]:
# If you need to update the database, use the 'startyear' and 'endyear' parameters
# zip_util = zu.ZipCodeUtility(api_headers=api_headers, startyear=2022, endyear=2023)
zip_util = zu.ZipCodeUtility(api_headers=api_headers)
zip_util.get_all_zip_zbp()

In [19]:
# This is included since sector '00' isn't present in the 
zip_util.get_zip_zbp('00')

Getting industry data for industry: 00	year: 2012
Finished processing 00 for 2012.
Getting industry data for industry: 00	year: 2013
Finished processing 00 for 2013.
Getting industry data for industry: 00	year: 2014
Finished processing 00 for 2014.
Getting industry data for industry: 00	year: 2015
Finished processing 00 for 2015.
Getting industry data for industry: 00	year: 2016
Finished processing 00 for 2016.
Getting industry data for industry: 00	year: 2017
Finished processing 00 for 2017.
Getting industry data for industry: 00	year: 2018
Finished processing 00 for 2018.
Getting industry data for industry: 00	year: 2019
Finished processing 00 for 2019.
Getting industry data for industry: 00	year: 2020
Finished processing 00 for 2020.
Getting industry data for industry: 00	year: 2021
Finished processing 00 for 2021.
Getting industry data for industry: 00	year: 2022
Failed to fetch data: 404
Failed to fetch data for year 2022: 404


# DimZipCode
## The below is for populating the 'DimZipCode' table, which will have the zipcode, city, and state columns

In [ ]:
def table_exists(conn, table_name):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
    exists = cursor.fetchone() is not None
    return exists

def get_all_zipcode_names(year):
    url = f"https://api.census.gov/data/{year}/zbp?get=GEO_TTL&for=zip%20code:*&key=975f39a54e48438ceebf303d6018e34db212e804"
    if int(year) == 2017 or int(year) == 2018:
        url = f"https://api.census.gov/data/{year}/zbp?get=NAME&for=zip%20code:*&key=975f39a54e48438ceebf303d6018e34db212e804"
    
    if int(year) > 2018:
        url = f"https://api.census.gov/data/{year}/cbp?get=NAME&for=zip%20code:*&key=975f39a54e48438ceebf303d6018e34db212e804"
    
    response = r.get(url)
    if response.status_code == 200:
        data = response.json()
        # Skip the header row and extract the relevant data
        return [(row[1], row[0]) for row in data[1:]]
    else:
        print(f"Failed to fetch data for year {year}: {response.status_code}")
        return []

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS ZipCodeNames')
if not table_exists(conn, 'DimZipCode'):
    query = "SELECT DISTINCT GeoID FROM DataEntry"
    cursor.execute(query)
    geo_ids = cursor.fetchall()
    geo_id_set = set(geo_id[0] for geo_id in geo_ids)

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DimZipCode (
        GeoID TEXT PRIMARY KEY,
        GeoName TEXT
    )
    ''')
    cursor.execute("PRAGMA table_info(DimZipCode);")
    columns = [info[1] for info in cursor.fetchall()]
    if 'GeoName' in columns:
        # Fetch and store names for each year from 2012 to 2022
        unique_zip_code_data = set()
        for year in tqdm(range(2012, endyear), desc="Fetching zip code names"):
            zip_code_data = get_all_zipcode_names(str(year))
            for geo_id, geo_name in zip_code_data:
                if geo_id in geo_id_set:
                    unique_zip_code_data.add((geo_id, geo_name))

        # Insert the unique data into the database
        cursor.executemany('INSERT OR IGNORE INTO DimZipCode (GeoID, GeoName) VALUES (?, ?)', list(unique_zip_code_data))
        conn.commit()
conn.close()

In [112]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Alter table to add City and State columns
try:
    cursor.execute("PRAGMA table_info(DimZipCode);")
    columns = [info[1] for info in cursor.fetchall()]
    
    # Add City column if it does not exist
    if 'City' not in columns:
        cursor.execute("ALTER TABLE DimZipCode ADD COLUMN City TEXT;")
        print("Column 'City' added successfully.")
    
    # Add State column if it does not exist
    if 'State' not in columns:
        cursor.execute("ALTER TABLE DimZipCode ADD COLUMN State TEXT;")
        print("Column 'State' added successfully.")
except sqlite3.OperationalError as e:
    print(f"Error: {e}")

# Commit the changes and close the connection
conn.commit()
conn.close()


In [113]:
# Function to extract city and state from GeoName
def extract_city_state(geo_name):
    match = re.match(r'ZIP \d+ \((.+), (.+)\)', geo_name)
    if match:
        return match.group(1), match.group(2)
    else:
        return None, None

if 'GeoName' in columns:
    conn = sqlite3.connect('zip_data/us_economic_data.db')
    cursor = conn.cursor()
    cursor.execute("SELECT GeoID, GeoName FROM DimZipCode")
    rows = cursor.fetchall()

    # Prepare the data for updating
    update_data = []
    for geo_id, geo_name in tqdm(rows, desc="Extracting city and state"):
        city, state = extract_city_state(geo_name)
        if city and state:
            update_data.append((city, state, geo_id))

    # Update the table with city and state
    cursor.executemany("UPDATE DimZipCode SET City = ?, State = ? WHERE GeoID = ?", update_data)

    conn.commit()
    conn.close()


In [114]:
if 'GeoName' in columns:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Create a new table without the GeoName column
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS ZipCodeNames_new (
        GeoID TEXT PRIMARY KEY,
        City TEXT,
        State TEXT
    )
    ''')

    # Copy data from the old table to the new table
    cursor.execute('''
    INSERT INTO ZipCodeNames_new (GeoID, City, State)
    SELECT GeoID, City, State
    FROM DimZipCode
    ''')

    # Drop the old table
    cursor.execute('DROP TABLE DimZipCode')

    # Rename the new table to the original table name
    cursor.execute('ALTER TABLE ZipCodeNames_new RENAME TO DimZipCode')

    # Commit the changes and close the connection
    conn.commit()
    conn.close()


## This is for creating foreign keys to the existing 'DataEntry' Tables. Unfortunately, they were not included in the database_manager.py file, so I'm adding them here.

In [116]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(f"PRAGMA foreign_key_list(DataEntry)")
foreign_keys = cursor.fetchall()
if not foreign_keys:
    # Create the DimNaics and DimYear tables if they don't already exist

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DimYear (
        Year INTEGER PRIMARY KEY,
        YearDescription TEXT
    )
    ''')

    # Create the new DataEntry table with foreign keys
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DataEntry_new (
        EntryID INTEGER PRIMARY KEY AUTOINCREMENT,
        GeoID TEXT,  
        NaicsCode TEXT,  
        Year INTEGER,  
        Establishments INTEGER,
        Employees INTEGER,
        Payroll INTEGER,
        IndustryLevel INT,
        FOREIGN KEY (GeoID) REFERENCES DimZipCode(GeoID),
        FOREIGN KEY (NaicsCode) REFERENCES DimNaics(NaicsCode),
        FOREIGN KEY (Year) REFERENCES DimYear(Year)
    )
    ''')

    # Copy data from the old table to the new table
    cursor.execute('''
    INSERT INTO DataEntry_new (EntryID, GeoID, NaicsCode, Year, Establishments, Employees, Payroll, IndustryLevel)
    SELECT EntryID, GeoID, NaicsCode, Year, Establishments, Employees, Payroll, IndustryLevel
    FROM DataEntry
    ''')

    # Drop the old table
    cursor.execute('DROP TABLE DataEntry')

    # Rename the new table to DataEntry
    cursor.execute('ALTER TABLE DataEntry_new RENAME TO DataEntry')

    conn.commit()
conn.close()


# DimNaics Table

### This table has the naics industry ids along with the names of the industries.

In [126]:
# Load the CSV file
df = pd.read_csv('id_lists/industry_id_list.csv')

# Function to format relevant_naics values
def format_naics(value):
    if value == 0.0:
        return '00'
    else:
        return str(int(value)).zfill(2)

# Apply the formatting function to the relevant_naics column
df['relevant_naics'] = df['relevant_naics'].apply(format_naics)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create the table
cursor.execute('DROP TABLE DimNaics')
cursor.execute('''
CREATE TABLE IF NOT EXISTS DimNaics (
    NaicsCode TEXT,
    industry_detail TEXT
)
''')

# Delete existing data in the table to prevent duplications
cursor.execute('DELETE FROM DimNaics')
# Insert the data into the table
for index, row in df.iterrows():
    cursor.execute('INSERT INTO DimNaics (NaicsCode, industry_detail) VALUES (?, ?)', (row['relevant_naics'], row['industry_detail']))

conn.commit()
conn.close()


      Unnamed: 0 relevant_naics                             industry_detail
0              0             11  Agriculture, Forestry, Fishing and Hunting
1              1            111                             Crop Production
2              2           1111                   Oilseed and Grain Farming
3              3          11111                             Soybean Farming
4              4         111110                             Soybean Farming
...          ...            ...                                         ...
2211        2211         928110                          National Security 
2212        2212          92812                      International Affairs 
2213        2213         928120                      International Affairs 
2214        2214             99                   Industries not classified
2215        2215             00                       Total for all sectors

[2216 rows x 3 columns]


# DimYear Table

### This table has the years along with the corresponding NAICS year identifiers. For example, 'NAICS2017' is used for 2017 onwards.

In [128]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Function to select NAICS year description based on the given year
def _naics_year_selector(year):
    if year >= 2000 and year <= 2002:
        return "NAICS1997"
    elif year >= 2003 and year <= 2007:
        return "NAICS2002"
    elif year >= 2008 and year <= 2011:
        return "NAICS2007"
    elif year >= 2012 and year <= 2016:
        return "NAICS2012"
    return "NAICS2017"

# Populate the DimYear table with years and their NAICS descriptions
start_year = 2000
years_data = [(year, _naics_year_selector(year)) for year in range(start_year, endyear + 1)]

# Insert the data into the DimYear table
cursor.executemany('INSERT OR IGNORE INTO DimYear (Year, YearDescription) VALUES (?, ?)', years_data)

conn.commit()
conn.close()

# Check

## The cells below check if the database was populated properly

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
query = "SELECT * FROM DimZipCode"
cursor.execute(query)
rows = cursor.fetchall()
columns = [description[0] for description in cursor.description]
print(columns)
for row in rows:
    print(row)
conn.close()
len(rows)

In [6]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
query = "SELECT * FROM DimYear"
cursor.execute(query)
rows = cursor.fetchall()
columns = [description[0] for description in cursor.description]
print(columns)
for row in rows:
    print(row)
conn.close()
len(rows)

['Year', 'YearDescription']
(2000, 'NAICS1997')
(2001, 'NAICS1997')
(2002, 'NAICS1997')
(2003, 'NAICS2002')
(2004, 'NAICS2002')
(2005, 'NAICS2002')
(2006, 'NAICS2002')
(2007, 'NAICS2002')
(2008, 'NAICS2007')
(2009, 'NAICS2007')
(2010, 'NAICS2007')
(2011, 'NAICS2007')
(2012, 'NAICS2012')
(2013, 'NAICS2012')
(2014, 'NAICS2012')
(2015, 'NAICS2012')
(2016, 'NAICS2012')
(2017, 'NAICS2017')
(2018, 'NAICS2017')
(2019, 'NAICS2017')
(2020, 'NAICS2017')
(2021, 'NAICS2017')
(2022, 'NAICS2017')
(2023, 'NAICS2017')


24

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
query = "SELECT * FROM DimNaics"
cursor.execute(query)
rows = cursor.fetchall()
columns = [description[0] for description in cursor.description]
print(columns)
for row in rows:
    print(row)
conn.close()
len(rows)

In [131]:
def count_rows_in_table(table_name):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        conn.close()
        return count
    except sqlite3.Error as e:
        print(f"Error counting rows in {table_name}: {e}")
        conn.close()
        return None
count_rows_in_table('DataEntry')

36194419

In [133]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Function to print foreign keys of a table
def print_foreign_keys(table_name):
    cursor.execute(f"PRAGMA foreign_key_list({table_name})")
    foreign_keys = cursor.fetchall()
    if foreign_keys:
        print(f"Foreign keys in table {table_name}:")
        for key in foreign_keys:
            print(key)
    else:
        print(f"No foreign keys in table {table_name}.")

# Check foreign keys for the DataEntry table
print_foreign_keys('DataEntry')

# Close the connection
conn.close()

Foreign keys in table DataEntry:
(0, 0, 'DimYear', 'Year', 'Year', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'DimZipCode', 'GeoID', 'GeoID', 'NO ACTION', 'NO ACTION', 'NONE')
